In [17]:
dic = {}
dic['us-south-central'] = '13.65.92.139'
dic['us-west1'] = "13.93.236.162"
dic['us-central'] = "localhost"
dic['asia-japan-east'] = "13.78.83.9"
dic['eu-north-europe'] = '52.178.201.184'
dic['eu-uk-west'] = '51.141.11.143'
dic['asia-japan-west'] = '104.214.146.200'
def get_percentile(file_name, percentile):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                if len(l) == 0 or len(l) == 1:
                    continue

                d[l[0]].append(int(l[1]))
    dic = {}
    import numpy as np
    for key, value in d.iteritems():
        a = np.array(value)
#         print np.amax(a)
        p = np.percentile(a, percentile)/1000
        dic[key] = p
    return dic

def data_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Put_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(fp + storage_name + "_storage_server.log", percentile)['Put_Azure_' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)
    
def metadata_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Accept_Phase_thrfit_Table_' + storage_name]
    azure_latency = get_percentile(fp + storage_name + "_table_server.log", percentile)['Accept_Phase_AzureTable_' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)

def get_data_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['GET_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(fp + storage_name + "_storage_server.log", percentile)['Get_Azure_' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)
    
def get_metadata_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Get_Highest_Version_Thrift_Table_' + storage_name]
    azure_latency = get_percentile(fp + storage_name + "_table_server.log", percentile)['Get_Highest_Version_Number_AzureTable_' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)



In [16]:
def put_result(fp, percentile, size, three_dc):
    f = fp + 'performance_log/giza_latency_put_' + size + '.txt.log'
    print 'Put              : ' + str(get_percentile(f, percentile)['Put'])
    print 'Data Path     Put: ' + str(get_percentile(f, percentile)['SendErasureEncode'])
    print three_dc[0] + '       Put: ' + data_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '      Put: ' + data_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '         Put: ' + data_breakdown_row(fp, three_dc[2], percentile)
    print '----------------------------------------------------------'
    get_version_latency = get_percentile(f, percentile)['GetLocalHighestCommittedVersionNumber']
    fast_paxos_latency = get_percentile(f, percentile)['DoPhase2']
    print 'Metadata Path Put: ' + str(get_version_latency + fast_paxos_latency)
    print 'Get Highest Version: ' + str(get_version_latency)
    print 'Do Fast Paxos     : ' + str(fast_paxos_latency)
    print three_dc[0] + '        Put: ' + metadata_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '       Put: ' + metadata_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '          Put: ' + metadata_breakdown_row(fp, three_dc[2], percentile)

def get_result(fp, percentile, size, three_dc):
    f = fp + 'performance_log/giza_latency_get_' + size + '.txt.log'
    print 'Get:           ' + str(get_percentile(f, percentile)['mainGet'])
    print 'Data Path Get: ' + str(get_percentile(f, percentile)['GetErasureEncode'])
    print three_dc[0] + '       Get: ' + get_data_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '      Get: ' + get_data_breakdown_row(fp, three_dc[1], percentile)
    print '----------------------------------------------------------'
    get_version_latency = get_percentile(f, percentile)['GetLocal']
    print 'Get Local Highest Version: ' + str(get_version_latency)
    print 'Get Global Highest Version: ' + str(get_percentile(f, percentile)['GetGlobalHighestCommittedVersion'])
    print three_dc[0] + '        Get: ' + get_metadata_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '       Get: ' + get_metadata_breakdown_row(fp, three_dc[1], percentile)


In [14]:
def metadata_breakdown_row(fp, storage_name, percentile):
#     thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Accept_Phase_thrfit_Table_' + storage_name]
    print get_percentile(fp + storage_name + "_table_server.log", percentile)
    azure_latency = get_percentile(fp + storage_name + "_table_server.log", percentile)['Accept_Phase_AzureTable_' + storage_name]
#     difference = thrift_latency - azure_latency
    return azure_latency
three_dc = ['eu-north-europe', 'us-central', 'asia-japan-east']
fp = '4mb/put/'
l = [] 
p = 50
for dc in three_dc:
    l.append(metadata_breakdown_row(fp, dc, p))

{}


KeyError: 'Accept_Phase_AzureTable_eu-north-europe'

In [18]:
# 2-1 US 4mb
three_dc = ['us-central', 'eu-north-europe', 'asia-japan-east']
put_result('4mb/put/', 50, '4mb', three_dc)
print '###########################################################'
put_result('2mb/put/', 50, '2mb', three_dc)
print '###########################################################'
put_result('1mb/put/', 50, '1mb', three_dc)
print '###########################################################'
put_result('256kb/put/', 50, '256kb', three_dc)
print '###########################################################'

Put              : 344.339
Data Path     Put: 343.64
us-central       Put: 35.5265  |  33.704  |  1.8225
eu-north-europe      Put: 259.63  |  57.246  |  202.384
asia-japan-east         Put: 305.143  |  31.76  |  273.383
----------------------------------------------------------
Metadata Path Put: 188.653
Get Highest Version: 65.385
Do Fast Paxos     : 123.268
us-central        Put: 64.9425  |  64.811  |  0.1315


KeyError: 'Accept_Phase_thrfit_Table_eu-north-europe'

In [22]:
get_result('4mb/get/', 95, '4mb', three_dc)
print '#############################################################'
get_result('2mb/get/', 95, '2mb', three_dc)
print '#############################################################'
get_result('1mb/get/', 95, '1mb', three_dc)
print '#############################################################'
get_result('256kb/get/', 95, '256kb', three_dc)
print '#############################################################'

Get:               348.3411
Data Path     Get: 319.27565
us-central       Get: 83.56285  |  80.32705  |  3.2358
eu-north-europe      Get: 315.4111  |  205.3346  |  110.0765
----------------------------------------------------------
Get Local Highest Version: 27.8263
Get Global Highest Version: 118.3087
us-central        Get: 14.62315  |  14.4739  |  0.14925
eu-north-europe       Get: 118.05315  |  15.58015  |  102.473
#############################################################
Get:               309.26915
Data Path     Get: 281.21195
us-central       Get: 72.4401  |  71.21915  |  1.22095
eu-north-europe      Get: 279.7214  |  169.69395  |  110.02745
----------------------------------------------------------
Get Local Highest Version: 35.17985
Get Global Highest Version: 121.5108
us-central        Get: 17.40555  |  17.3083  |  0.09725
eu-north-europe       Get: 121.08305  |  19.00175  |  102.0813
#############################################################
Get:               360.894


In [ ]:
def get_std(file_name, percentile):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                d[l[0]].append(int(l[1]))
    dic = {}
    import numpy as np
    for key, value in d.iteritems():
        a = np.array(value)
#         print np.amax(a)
        p = np.std(a, percentile)/1000
        dic[key] = p
    return dic

In [23]:
def get_list(file_name):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                if len(l) == 0 or len(l) == 1:
                    continue

                d[l[0]].append(int(l[1]))
    return d

In [37]:
import numpy as np
a = get_list('4mb/put/eu-north-europe_table_server.log')
print np.mean(a['Accept_Phase_AzureTable_eu-north-europe'])
print np.std(a['Accept_Phase_AzureTable_eu-north-europe'])

74406.5214785
16780.6758329


In [4]:
def put_result_seven(fp, percentile, size, three_dc):
    print len(three_dc)
    f = fp + 'performance_log/giza_latency_put_' + size + '.txt.log'
    print 'Put              : ' + str(get_percentile(f, percentile)['Put'])
    print 'Data Path     Put: ' + str(get_percentile(f, percentile)['SendErasureEncode'])
    print three_dc[0] + '       Put: ' + data_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '      Put: ' + data_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '         Put: ' + data_breakdown_row(fp, three_dc[2], percentile)
    print three_dc[3] + '         Put: ' + data_breakdown_row(fp, three_dc[3], percentile)
    print three_dc[4] + '         Put: ' + data_breakdown_row(fp, three_dc[4], percentile)
    print three_dc[5] + '         Put: ' + data_breakdown_row(fp, three_dc[5], percentile)
    print three_dc[6] + '         Put: ' + data_breakdown_row(fp, three_dc[6], percentile)
    print '----------------------------------------------------------'
    get_version_latency = get_percentile(f, percentile)['GetLocalHighestCommittedVersionNumber']
    fast_paxos_latency = get_percentile(f, percentile)['DoPhase2']
    print 'Metadata Path Put: ' + str(get_version_latency + fast_paxos_latency)
    print 'Get Highest Version: ' + str(get_version_latency)
    print 'Do Fast Paxos     : ' + str(fast_paxos_latency)
    print three_dc[0] + '        Put: ' + metadata_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '       Put: ' + metadata_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '          Put: ' + metadata_breakdown_row(fp, three_dc[2], percentile)

def get_result_seven(fp, percentile, size, three_dc):
    f = fp + 'performance_log/giza_latency_get_' + size + '.txt.log'
    print 'Get:               ' + str(get_percentile(f, percentile)['mainGet'])
    print 'Data Path     Get: ' + str(get_percentile(f, percentile)['GetErasureEncode'])
    print three_dc[0] + '       Get: ' + get_data_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '      Get: ' + get_data_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '      Get: ' + get_data_breakdown_row(fp, three_dc[2], percentile)
    print three_dc[3] + '      Get: ' + get_data_breakdown_row(fp, three_dc[3], percentile)
    print three_dc[4] + '      Get: ' + get_data_breakdown_row(fp, three_dc[4], percentile)
    print three_dc[5] + '      Get: ' + get_data_breakdown_row(fp, three_dc[5], percentile)
    print '----------------------------------------------------------'
    get_version_latency = get_percentile(f, percentile)['GetLocal']
    print 'Get Local Highest Version: ' + str(get_version_latency)
    print 'Get Global Highest Version: ' + str(get_percentile(f, percentile)['GetGlobalHighestCommittedVersion'])
    print three_dc[0] + '        Get: ' + get_metadata_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '       Get: ' + get_metadata_breakdown_row(fp, three_dc[1], percentile)

In [32]:
def put_result_seven(fp, percentile, size, three_dc):
    print len(three_dc)
    f = fp + 'performance_log/giza_latency_put_' + size + '.txt.log'
    print 'Put              : ' + str(get_percentile(f, percentile)['Put'])
    print 'Data Path     Put: ' + str(get_percentile(f, percentile)['SendErasureEncode'])
    print three_dc[0] + '       Put: ' + data_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '      Put: ' + data_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '         Put: ' + data_breakdown_row(fp, three_dc[2], percentile)
    print three_dc[3] + '         Put: ' + data_breakdown_row(fp, three_dc[3], percentile)
    print three_dc[4] + '         Put: ' + data_breakdown_row(fp, three_dc[4], percentile)
    print three_dc[5] + '         Put: ' + data_breakdown_row(fp, three_dc[5], percentile)
    print three_dc[6] + '         Put: ' + data_breakdown_row(fp, three_dc[6], percentile)
    print '----------------------------------------------------------'
    get_version_latency = get_percentile(f, percentile)['GetLocalHighestCommittedVersionNumber']
    fast_paxos_latency = get_percentile(f, percentile)['DoPhase2']
    print 'Metadata Path Put: ' + str(get_version_latency + fast_paxos_latency)
    print 'Get Highest Version: ' + str(get_version_latency)
    print 'Do Fast Paxos     : ' + str(fast_paxos_latency)
    print three_dc[0] + '        Put: ' + metadata_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '       Put: ' + metadata_breakdown_row(fp, three_dc[1], percentile)
    print three_dc[2] + '          Put: ' + metadata_breakdown_row(fp, three_dc[2], percentile)

three_dc = ['us-central', 'us-south-central', 'us-west1', 'eu-north-europe', 'eu-uk-west', 'asia-japan-east', 'asia-japan-west']
put_result_seven('4mb/put/', 95, '4mb', three_dc)
print '#############################################'

7
Put              : 413.539
Data Path     Put: 409.045
us-central       Put: 58.21635  |  56.336  |  1.88035
us-south-central      Put: 178.59095  |  127.525  |  51.06595
us-west1         Put: 190.99295  |  108.374  |  82.61895
eu-north-europe         Put: 280.99585  |  81.121  |  199.87485
eu-uk-west         Put: 249.0303  |  27.726  |  221.3043
asia-japan-east         Put: 330.1205  |  74.639  |  255.4815
asia-japan-west         Put: 340.6325  |  89.308  |  251.3245
----------------------------------------------------------
Metadata Path Put: 294.231
Get Highest Version: 81.552
Do Fast Paxos     : 212.679
us-central        Put: 80.2503  |  80.494  |  -0.2437
us-south-central       Put: 179.57445  |  156.723  |  22.85145
us-west1          Put: 199.6127  |  162.236  |  37.3767
#############################################


In [5]:
three_dc = ['us-central', 'us-south-central', 'us-west1', 'eu-north-europe', 'eu-uk-west', 'asia-japan-east', 'asia-japan-west']
put_result_seven('4mb/put/', 95, '4mb', three_dc)
print '#############################################################'
put_result_seven('2mb/put/', 95, '2mb', three_dc)
print '#############################################################'
put_result_seven('1mb/put/', 95, '1mb', three_dc)
print '#############################################################'
put_result_seven('256kb/put/', 95, '256kb', three_dc)
print '#############################################################'

7
Put              : 413.539
Data Path     Put: 409.045
us-central       Put: 58.21635  |  56.336  |  1.88035
us-south-central      Put: 178.59095  |  127.525  |  51.06595
us-west1         Put: 190.99295  |  108.374  |  82.61895
eu-north-europe         Put: 280.99585  |  81.121  |  199.87485
eu-uk-west         Put: 249.0303  |  27.726  |  221.3043
asia-japan-east         Put: 330.1205  |  74.639  |  255.4815
asia-japan-west         Put: 340.6325  |  89.308  |  251.3245
----------------------------------------------------------
Metadata Path Put: 294.231
Get Highest Version: 81.552
Do Fast Paxos     : 212.679
us-central        Put: 80.2503  |  80.494  |  -0.2437
us-south-central       Put: 179.57445  |  156.723  |  22.85145
us-west1          Put: 199.6127  |  162.236  |  37.3767
#############################################################
7
Put              : 346.6224
Data Path     Put: 338.8735
us-central       Put: 55.18365  |  54.2925  |  0.89115
us-south-central      Put: 158.1812 

In [6]:
get_result_seven('4mb/get/', 95, '4mb', three_dc)
print '#############################################################'
get_result_seven('2mb/get/', 95, '2mb', three_dc)
print '#############################################################'
get_result_seven('1mb/get/', 95, '1mb', three_dc)
print '#############################################################'
get_result_seven('256kb/get/', 95, '256kb', three_dc)

Get:               339.80175
Data Path     Get: 317.5537
us-central       Get: 63.7304  |  62.86125  |  0.86915
us-south-central      Get: 184.93505  |  155.7241  |  29.21095
us-west1      Get: 178.8976  |  128.3861  |  50.5115
eu-north-europe      Get: 241.03905  |  127.548  |  113.49105
eu-uk-west      Get: 153.4846  |  34.17675  |  119.30785
asia-japan-east      Get: 229.17525  |  71.8923  |  157.28295
----------------------------------------------------------
Get Local Highest Version: 30.68395
Get Global Highest Version: 67.61015
us-central        Get: 13.4501  |  13.3551  |  0.095
us-south-central       Get: 63.28305  |  39.50115  |  23.7819
#############################################################
Get:               404.20815
Data Path     Get: 374.18305
us-central       Get: 54.23315  |  53.47815  |  0.755
us-south-central      Get: 266.16065  |  218.275  |  47.88565
us-west1      Get: 197.92905  |  130.38025  |  67.5488
eu-north-europe      Get: 304.55385  |  185.6086  |  

In [7]:
get_result_seven('4mb/get/', 95, '4mb', three_dc)
print '#############################################################'
get_result_seven('2mb/get/', 95, '2mb', three_dc)
print '#############################################################'
get_result_seven('1mb/get/', 95, '1mb', three_dc)
print '#############################################################'
get_result_seven('256kb/get/', 95, '256kb', three_dc)

Get:               339.80175
Data Path     Get: 317.5537
us-central       Get: 63.7304  |  62.86125  |  0.86915
us-south-central      Get: 184.93505  |  155.7241  |  29.21095
us-west1      Get: 178.8976  |  128.3861  |  50.5115
eu-north-europe      Get: 241.03905  |  127.548  |  113.49105
eu-uk-west      Get: 153.4846  |  34.17675  |  119.30785
asia-japan-east      Get: 229.17525  |  71.8923  |  157.28295
----------------------------------------------------------
Get Local Highest Version: 30.68395
Get Global Highest Version: 67.61015
us-central        Get: 13.4501  |  13.3551  |  0.095
us-south-central       Get: 63.28305  |  39.50115  |  23.7819
#############################################################
Get:               404.20815
Data Path     Get: 374.18305
us-central       Get: 54.23315  |  53.47815  |  0.755
us-south-central      Get: 266.16065  |  218.275  |  47.88565
us-west1      Get: 197.92905  |  130.38025  |  67.5488
eu-north-europe      Get: 304.55385  |  185.6086  |  